Importing Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.formula.api as smf
%matplotlib inline
import seaborn as sns

Reading CSV files of play by play data downloaded from NFLScrapR

In [2]:
pbp_2017 = pd.read_csv("/Users/maxsteinberg/Desktop/ms_data_analytics/pbp_2017.csv")
pbp_2018 = pd.read_csv("/Users/maxsteinberg/Desktop/ms_data_analytics/pbp_2018.csv")
pbp_2019 = pd.read_csv("/Users/maxsteinberg/Desktop/ms_data_analytics/pbp_2019.csv")

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (168,169,170,171,176,177,180,181,184,185,190,191,192,193,205,206,207,220,221,222,223,224,225,228,229,233,234,235,240,242,243) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Concatenating 2017 and 2018 play_by_play data for model training

In [3]:
dfs = [pbp_2017, pbp_2018]
all_pbp = pd.concat(dfs).reset_index(drop=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Set up to make coding easier

In [4]:
pd.set_option('display.max_columns', None)
all_pbp.columns = [i.replace(' ', '_').lower() for i in all_pbp.columns]
all_pbp.head()

,absscorediff,accepted.penalty,airyards,awayteam,awaytimeouts_remaining_post,awaytimeouts_remaining_pre,away_wp_post,away_wp_pre,blockingplayer,chalreplayresult,challenge.replay,date,defteamscore,deftwopoint,defensiveteam,drive,epa,expointresult,expoint_prob,exppts,fieldgoaldistance,fieldgoalresult,field_goal_prob,firstdown,fumble,gameid,goaltogo,hometeam,hometimeouts_remaining_post,hometimeouts_remaining_pre,home_wp_post,home_wp_pre,interceptionthrown,interceptor,no_score_prob,onsidekick,opp_field_goal_prob,opp_safety_prob,opp_touchdown_prob,passattempt,passlength,passlocation,passoutcome,passer,passer_id,penalizedplayer,penalizedteam,penalty.yards,penaltytype,playattempted,playtimediff,playtype,posteamscore,puntresult,qbhit,recfumbplayer,recfumbteam,receiver,receiver_id,reception,returnresult,returner,rungap,runlocation,rushattempt,rusher,rusher_id,sack,safety,safety_prob,scorediff,season,sideoffield,tackler1,tackler2,timesecs,timeunder,timeout_indicator,timeout_team,touchdown,touchdown_prob,twopointconv,twopoint_prob,unnamed:_0,wpa,win_prob,yards.gained,yardsaftercatch,airepa,airwpa,desc,down,play_id,posteam,posteam_timeouts_pre,qtr,sp,time,yacepa,yacwpa,ydsnet,ydstogo,yrdline100,yrdln
0,0.0,0,0,KC,3,3,0.493982,0.499993,NaN,NaN,0,2017-09-07,0.0,NaN,KC,1,0.194396,NaN,0.0,0.814998,NaN,NaN,0.233081,NaN,0,2017090700,0.0,NE,3,3,0.506018,0.500007,0,NaN,0.001374,0,0.162632,0.004441,0.254179,0,NaN,NaN,NaN,NaN,None,NaN,NaN,0,NaN,1,0.0,Kickoff,0.0,NaN,0,NaN,NaN,NaN,None,0,NaN,D.Lewis,NaN,NaN,0,NaN,None,0,0,0.003656,0.0,2017,KC,K.Pierre-Louis,D.Harris,3600.0,15,0,None,0,0.340639,NaN,0.0,NaN,0.006011,0.500007,26,0,NaN,NaN,C.Santos kicks 64 yards from KC 35 to NE 1. D....,NaN,44,NE,3,1,0,15:00,NaN,NaN,73,0,35.0,35.0
1,0.0,0,27,KC,3,3,0.515945,0.493982,NaN,NaN,0,2017-09-07,0.0,NaN,KC,1,-0.764068,NaN,0.0,1.009395,NaN,NaN,0.240728,0.0,0,2017090700,0.0,NE,3,3,0.484055,0.506018,0,NaN,0.001347,0,0.155819,0.003770,0.243413,1,Deep,left,Incomplete Pass,T.Brady,00-0019596,NaN,NaN,0,NaN,1,5.0,Pass,0.0,NaN,0,NaN,NaN,D.Allen,00-0029689,0,NaN,NaN,NaN,NaN,0,NaN,None,0,0,0.003671,0.0,2017,NE,NaN,NaN,3595.0,15,0,None,0,0.351251,NaN,0.0,NaN,-0.021963,0.506018,0,0,2.186206,0.068249,(14:55) NE 12-Brady 18th season as Patriots QB...,1.0,68,NE,3,1,0,14:55,-2.950274,-0.090213,0,10,73.0,27.0
2,0.0,0,1,KC,3,3,0.489990,0.515945,NaN,NaN,0,2017-09-07,0.0,NaN,KC,1,0.816847,NaN,0.0,0.245327,NaN,NaN,0.222335,0.0,0,2017090700,0.0,NE,3,3,0.510010,0.484055,0,NaN,0.001421,0,0.181320,0.004977,0.284096,1,Short,right,Complete,T.Brady,00-0019596,NaN,NaN,0,NaN,1,6.0,Pass,0.0,NaN,0,NaN,NaN,R.Burkhead,00-0030288,1,NaN,NaN,NaN,NaN,0,NaN,None,0,0,0.004009,0.0,2017,NE,R.Parker,NaN,3589.0,15,0,None,0,0.301842,NaN,0.0,NaN,0.025955,0.484055,8,7,-0.848627,-0.024329,(14:49) T.Brady pass short right to R.Burkhead...,2.0,94,NE,3,1,0,14:49,1.665474,0.050284,8,10,73.0,27.0
3,0.0,0,0,KC,3,3,0.447018,0.489990,NaN,NaN,0,2017-09-07,0.0,NaN,KC,1,1.369456,NaN,0.0,1.062174,NaN,NaN,0.213148,1.0,0,2017090700,0.0,NE,3,3,0.552982,0.510010,0,NaN,0.001426,0,0.160903,0.003032,0.244357,0,NaN,NaN,NaN,NaN,None,NaN,NaN,0,NaN,1,35.0,Run,0.0,NaN,0,NaN,NaN,NaN,None,0,NaN,NaN,guard,left,1,J.White,00-0031062,0,0,0.003585,0.0,2017,NE,R.Parker,D.Johnson,3554.0,15,0,None,0,0.373548,NaN,0.0,NaN,0.042972,0.510010,8,0,NaN,NaN,(14:14) (Shotgun) J.White left guard to NE 43 ...,3.0,118,NE,3,1,0,14:14,NaN,NaN,73,2,65.0,35.0
4,0.0,0,0,KC,3,3,0.453025,0.447018,NaN,NaN,0,2017-09-07,0.0,NaN,KC,1,-0.238038,NaN,0.0,2.431630,NaN,NaN,0.297028,0.0,0,2017090700,0.0,NE,3,3,0.546975,0.552982,0,NaN,0.001206,0,0.105299,0.000968,0.163717,0,NaN,NaN,NaN,NaN,None,NaN,NaN,0,NaN,1,22.0,Run,0.0,NaN,0,NaN,NaN,NaN,None,0,NaN,NaN,NaN,middle,1,J.White,00-0031062,0,0,0.003613,0.0,2017,NE,E.Berry,NaN,3532.0,14,0,None,0,0.428168,NaN,0.0,NaN,-0.006007,0.552982,3,0,NaN,NaN,"(13:52) (No Huddle, Shotgun) J.White up the mi...",1.0,139,NE,3,1,0,13:52,NaN,NaN,19,10,57.0,43.0


Function to feature engineer new column which indicates whether or not a throw was an "End Zone Throw."
My definition of an end zone throw is a throw which travels in the air to inside the 1 yard line.

In [5]:
new_col = []
for _id, row in all_pbp.iterrows():
        if row['passattempt']==1 and (row['yrdline100'] - row['airyards']) <=1:
            new_col.append(1)
        else:
            new_col.append(0)
all_pbp['ez_throw'] = new_col

Removing data points inside of 60 seconds left in either half. Wanted to get rid of data that was essentially a QB throwing a hail mary. I'm more interested in on much a QB is throwing it to the end zone when it's not incredibly likely.

In [6]:
new_col = []
for _id, row in all_pbp.iterrows():
        if row['timesecs'] <= 60:
            new_col.append(1)
        elif row['timesecs'] > 1800 and row['timesecs'] <= 1860:
            new_col.append(1)
        else:
            new_col.append(0)
all_pbp['one_min_left'] = new_col
all_pbp = all_pbp[all_pbp['one_min_left']==0]

all_pbp = all_pbp.loc[(all_pbp['rushattempt']==1) | (all_pbp['passattempt']==1)]
all_pbp.ez_throw.value_counts()

0    57538
1     2277
Name: ez_throw, dtype: int64

Cleaning data for model fitting

In [7]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
all_pbp = all_pbp[all_pbp['yrdline100'].notnull()]
#gets rid of two point conversions
all_pbp = all_pbp[all_pbp['down'].notnull()]

First I'm importing the statsmodels logistic regression because I like the visual it provides. The actual model I'm training will use sklearn

In [8]:
import statsmodels.discrete.discrete_model as sm
feature_cols = ['yrdline100', 'scorediff']
X = all_pbp[feature_cols]
y = all_pbp.ez_throw
logreg.fit(X, y)
logit = sm.Logit(y, X)
mdl = logit.fit()

Optimization terminated successfully.
         Current function value: 0.106540
         Iterations 9


/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


Visual of the trained model. The r-squared is .34, this is essentially a measure of predictive power from 0-1. This number indicates to me that simply where a QB is on the field and the difference in score predicts the amount of end zone throws pretty well.

In [9]:
from scipy import stats
print(mdl.summary())

                           Logit Regression Results                           
Dep. Variable:               ez_throw   No. Observations:                59625
Model:                          Logit   Df Residuals:                    59623
Method:                           MLE   Df Model:                            1
Date:                Fri, 03 Jul 2020   Pseudo R-squ.:                  0.3414
Time:                        13:23:01   Log-Likelihood:                -6352.4
converged:                       True   LL-Null:                       -9645.3
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
yrdline100    -0.1121      0.001    -87.901      0.000      -0.115      -0.110
scorediff     -0.0189      0.002     -8.658      0.000      -0.023      -0.015


Checking what yrdline100 actually means. 95 is a teams own 5 yard line. So, our model is indicating the further away from the end zone we are, the less likely a QB is to throw it in the endzone. Obviously, this is intuitive.

In [10]:
all_pbp[all_pbp['yrdline100']==95].head()

,absscorediff,accepted.penalty,airyards,awayteam,awaytimeouts_remaining_post,awaytimeouts_remaining_pre,away_wp_post,away_wp_pre,blockingplayer,chalreplayresult,challenge.replay,date,defteamscore,deftwopoint,defensiveteam,drive,epa,expointresult,expoint_prob,exppts,fieldgoaldistance,fieldgoalresult,field_goal_prob,firstdown,fumble,gameid,goaltogo,hometeam,hometimeouts_remaining_post,hometimeouts_remaining_pre,home_wp_post,home_wp_pre,interceptionthrown,interceptor,no_score_prob,onsidekick,opp_field_goal_prob,opp_safety_prob,opp_touchdown_prob,passattempt,passlength,passlocation,passoutcome,passer,passer_id,penalizedplayer,penalizedteam,penalty.yards,penaltytype,playattempted,playtimediff,playtype,posteamscore,puntresult,qbhit,recfumbplayer,recfumbteam,receiver,receiver_id,reception,returnresult,returner,rungap,runlocation,rushattempt,rusher,rusher_id,sack,safety,safety_prob,scorediff,season,sideoffield,tackler1,tackler2,timesecs,timeunder,timeout_indicator,timeout_team,touchdown,touchdown_prob,twopointconv,twopoint_prob,unnamed:_0,wpa,win_prob,yards.gained,yardsaftercatch,airepa,airwpa,desc,down,play_id,posteam,posteam_timeouts_pre,qtr,sp,time,yacepa,yacwpa,ydsnet,ydstogo,yrdline100,yrdln,ez_throw,one_min_left
148,6.0,0,0,KC,3,3,0.273339,0.223787,NaN,NaN,0,2017-09-07,27.0,NaN,NE,20,1.481895,NaN,0.0,-1.028339,NaN,NaN,0.157479,0.0,0,2017090700,0.0,NE,3,3,0.726661,0.776213,0,NaN,0.063953,0,0.216528,0.019762,0.327981,0,NaN,NaN,NaN,NaN,None,NaN,NaN,0,NaN,1,0.0,Run,21.0,NaN,0,NaN,NaN,NaN,None,0,NaN,NaN,NaN,middle,1,K.Hunt,00-0033923,0,0,0.003176,-6.0,2017,KC,P.Chung,NaN,904.0,1,0,None,0,0.211121,NaN,0.0,NaN,0.049552,0.223787,13,0,NaN,NaN,(:04) (Shotgun) K.Hunt up the middle to KC 18 ...,1.0,3635,KC,3,3,0,00:04,NaN,NaN,8,15,95.0,5.0,0,0
345,9.0,0,0,NYJ,3,3,0.225243,0.216035,NaN,NaN,0,2017-09-10,12.0,NaN,NYJ,18,-0.400892,NaN,0.0,-1.727391,NaN,NaN,0.067792,0.0,0,2017091000,0.0,BUF,3,3,0.774757,0.783965,0,NaN,0.242237,0,0.219790,0.014829,0.315450,1,Short,left,Complete,T.Taylor,00-0028118,NaN,NaN,0,NaN,1,35.0,Pass,21.0,NaN,0,NaN,NaN,C.Clay,00-0028112,1,NaN,NaN,NaN,NaN,0,NaN,None,0,0,0.002581,9.0,2017,BUF,Ja.Adams,NaN,497.0,9,0,None,0,0.137321,NaN,0.0,NaN,-0.009208,0.783965,4,4,-0.762705,-0.027355,(8:17) (Shotgun) T.Taylor pass short left to C...,3.0,3650,BUF,3,4,0,08:17,0.361813,0.018146,5,9,95.0,5.0,0,0
603,3.0,0,0,OAK,2,2,0.622879,0.614083,NaN,NaN,0,2017-09-10,10.0,NaN,OAK,6,-0.255156,NaN,0.0,-0.845030,NaN,NaN,0.087849,0.0,0,2017091007,0.0,TEN,3,3,0.377121,0.385917,0,NaN,0.358757,0,0.159328,0.011662,0.233804,0,NaN,NaN,NaN,NaN,None,NaN,NaN,0,NaN,1,33.0,Run,7.0,NaN,0,NaN,NaN,NaN,None,0,NaN,NaN,guard,left,1,D.Henry,00-0032764,0,0,0.002170,-3.0,2017,TEN,E.Vanderdoes,NaN,2190.0,7,0,None,0,0.146431,NaN,0.0,NaN,-0.008796,0.385917,2,0,NaN,NaN,(6:30) D.Henry left guard to TEN 7 for 2 yards...,2.0,1293,TEN,3,2,0,06:30,NaN,NaN,29,7,95.0,5.0,0,0
1161,1.0,0,0,PIT,3,3,0.464577,0.463473,NaN,NaN,0,2017-09-10,7.0,NaN,CLE,11,0.077107,NaN,0.0,-0.399904,NaN,NaN,0.080260,0.0,0,2017091003,0.0,CLE,2,2,0.535423,0.536527,0,NaN,0.534655,0,0.105783,0.009538,0.156067,1,Short,middle,Complete,B.Roethlisberger,00-0022924,NaN,NaN,0,NaN,1,26.0,Pass,6.0,NaN,0,NaN,NaN,L.Bell,00-0030496,1,NaN,NaN,NaN,NaN,0,NaN,None,0,0,0.001534,-1.0,2017,PIT,J.Schobert,NaN,2007.0,4,0,None,0,0.112163,NaN,0.0,NaN,0.001103,0.463473,6,6,-0.337480,-0.011000,(3:27) (Shotgun) B.Roethlisberger pass short m...,1.0,1741,PIT,3,2,0,03:27,0.414586,0.012104,2,14,95.0,5.0,0,0
1366,3.0,0,6,ATL,2,2,0.585809,0.565163,NaN,NaN,0,2017-09-10,10.0,NaN,CHI,15,0.495005,NaN,0.0,-0.758205,NaN,NaN,0.145674,0.0,0,2017091001,0.0,CHI,2,2,0.414191,0.434837,0,NaN,0.106176,0,0.201076,0.018108,0.303178,1,Short,left,Complete,M.Ryan,00-0026143,NaN,NaN,0,NaN,1,10.0,Pass,13.0,NaN,0,NaN,NaN,M.Sanu,00-0029632,1,NaN,NaN,NaN,NaN,0,NaN,None,0,0,0.002809,3.0,2017,ATL,K.Fuller,NaN,774.0,13,0,None,0,0.222978,NaN,0.0,NaN,0.020646,0.565163,7,1,0.252989,0.009855,(12:54) M.Ryan pass short left to M.Sanu to AT...,1.0,2801,ATL,2,4,0,12:54,

This is an example of what the model pops out. If we are 5 yards away from the endzone and up 20, the probability that on a passing play the QB will pass it into the endzone is 78%.

In [11]:
samp = [(5, 20)]
print(logreg.predict_proba(samp))

[[0.7863952 0.2136048]]


Cleaning out 2019 data in the same way as we did on the training data.

In [12]:
pbp_2019.columns = [i.replace(' ', '_').lower() for i in pbp_2019.columns]
pbp_2019 = pbp_2019.loc[(pbp_2019['rush_attempt']==1) | (pbp_2019['pass_attempt']==1)]
pbp_2019 = pbp_2019[pbp_2019['yardline_100'].notnull()]
pbp_2019 = pbp_2019[pbp_2019['down'].notnull()]

pbp_2019.rename(columns={'game_seconds_remaining' : 'timesecs', 'score_differential' : 'scorediff'}, inplace=True)

new_col = []
for _id, row in pbp_2019.iterrows():
        if row['timesecs'] <= 60:
            new_col.append(1)
        elif row['timesecs'] > 1800 and row['timesecs'] <= 1860:
            new_col.append(1)
        else:
            new_col.append(0)
            
pbp_2019['one_min_left'] = new_col
pbp_2019 = pbp_2019[pbp_2019['one_min_left']==0]

Looking at run pass ratios in general in 2019

In [13]:
p = pbp_2019.groupby('posteam').pass_attempt.sum().reset_index()
r = pbp_2019.groupby('posteam').rush_attempt.sum().reset_index()
tot = pd.merge(p, r, on='posteam')
tot['rp_ratio'] = tot['rush_attempt']/tot['pass_attempt']
tot.sort_values(by='rp_ratio', ascending=False)

,posteam,pass_attempt,rush_attempt,rp_ratio
2,BAL,401.0,489.0,1.219451
19,MIN,403.0,412.0,1.022333
28,SF,419.0,426.0,1.016706
27,SEA,453.0,414.0,0.913907
13,IND,438.0,399.0,0.910959
3,BUF,459.0,403.0,0.877996
30,TEN,418.0,362.0,0.866029
24,OAK,439.0,375.0,0.854214
12,HOU,464.0,367.0,0.790948
8,DAL,492.0,383.0,0.778455


Making predictions on the 2019 data from our model

In [14]:
feature_cols = ['yardline_100', 'scorediff']
X_new = pbp_2019[feature_cols]
arr = logreg.predict_proba(X_new)

#function to extract probability of yes instead of probability of no.
lst = []
for each in arr:
    lst.append(each[-1])
    
pbp_2019['ez_throw_proba'] = lst

Feature engineering the end zone throw column fror the 2019 data

In [15]:
new_col = []
for _id, row in pbp_2019.iterrows():
        if row['pass_attempt']==1 and (row['yardline_100'] - row['air_yards']) <=1:
            new_col.append(1)
        else:
            new_col.append(0)
            
pbp_2019['ez_throw'] = new_col

Seattle was notoriously run heavy in 2019, but they had a strange caveat. They loved throwing it in the end zone. I examine the data below.

In [16]:
sea = pbp_2019.loc[((pbp_2019['posteam']=='SEA') & (pbp_2019['ez_throw']==1))]
sea = sea.sort_values(by='air_yards', ascending=False)

In [17]:
for each in sea.desc:
    print(each)

(1:36) (Shotgun) R.Wilson pass deep middle intended for T.Lockett INTERCEPTED by T.Hill [A.Donald] at LA -3. Touchback.
(8:15) R.Wilson pass deep right to DK.Metcalf for 40 yards, TOUCHDOWN.
(5:16) (Shotgun) R.Wilson pass incomplete deep middle to T.Lockett.
(6:03) R.Wilson pass deep right to M.Turner for 33 yards, TOUCHDOWN.
(8:48) (Shotgun) R.Wilson pass incomplete deep middle to D.Moore.
(7:22) (Shotgun) R.Wilson pass deep left to DK.Metcalf for 28 yards, TOUCHDOWN.
(2:22) (Shotgun) G.Fant reported in as eligible.  R.Wilson pass incomplete deep left to DK.Metcalf.
(11:33) (Shotgun) R.Wilson pass incomplete deep right to D.Moore (I.Oliver).
(1:21) (Shotgun) R.Wilson pass incomplete deep right to DK.Metcalf.
(3:35) (Shotgun) R.Wilson pass incomplete deep right to W.Dissly [M.Davenport].
(8:29) (Shotgun) R.Wilson pass deep right to T.Lockett for 19 yards, TOUCHDOWN.
(10:15) (Shotgun) R.Wilson pass deep right to T.Lockett for 19 yards, TOUCHDOWN.
(14:38) (Shotgun) R.Wilson pass incomple

Seattle's number is honestly absurd. The model is trained on both run and pass plays, meaning the expected end zone throws don't account for how pass or run heavy a team is. Seattle should be penalized for this, but ended up throwing it in the endzone 50 times on only 450 pass attempts. Contrast this with Tampa Bay, who passed the ball 550 times, and still had 7 less end zone throws than Seattle.

Is this a good strategy? Probably not.

In [24]:
non_throwaway_ez = pbp_2019[pbp_2019['receiver_player_id'].notnull()]
expected_ez_throw = pbp_2019.groupby('posteam').ez_throw_proba.sum().to_frame(name= 'exp_ez_throw').reset_index()
ez_throw_total = non_throwaway_ez.groupby('posteam').ez_throw.sum().to_frame(name= 'ez_throw_total').reset_index()
total = pd.merge(expected_ez_throw, ez_throw_total, on = 'posteam')
total['ez_throw_above_exp'] = total['ez_throw_total']/total['exp_ez_throw']
total.sort_values(by='ez_throw_above_exp', ascending=False)

,posteam,exp_ez_throw,ez_throw_total,ez_throw_above_exp
27,SEA,33.560295,50,1.489856
29,TB,34.625344,43,1.241865
11,GB,32.009903,39,1.218373
12,HOU,29.673088,35,1.179520
2,BAL,39.495544,42,1.063411
5,CHI,27.288700,28,1.026066
15,KC,32.652686,33,1.010637
10,DET,31.895796,32,1.003267
25,PHI,36.521846,34,0.930950
23,NYJ,20.861017,19,0.910790
